# ResNet18 CIFAR-100 Training

**Refactored notebook using training_lib**

Original functionality preserved with ~85% less code duplication.

In [ ]:
# Import the training library
import sys
sys.path.append('./training_lib')

from training_lib import (
    create_model, get_data_loader, UniversalTrainer,
    get_experiment_logger, visualize_samples, print_model_summary
)

# Show library info
from training_lib import info
info()

## Experiment Configuration

In [ ]:
# Experiment configuration - easily adjustable
CONFIG = {
    'model_name': 'resnet18',
    'dataset': 'cifar100', 
    'epochs': 200,
    'batch_size': 256,
    'learning_rate': 10e-3,
    'use_mlflow': True,
    'subset_size': None,  # Use full dataset
    'visualize': True
}

print("Experiment Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Model Creation & Data Loading

In [ ]:
# Initialize experiment logger
logger = get_experiment_logger("ResNet18", "CIFAR-100")
logger.experiment_start()

# Create ResNet18 model
model = create_model(CONFIG['model_name'], num_classes=100, pretrained=True)
print_model_summary(model, CONFIG['model_name'])

# Create data loader with automatic input size detection
train_loader = get_data_loader(
    CONFIG['dataset'],
    batch_size=CONFIG['batch_size'],
    subset_size=CONFIG['subset_size']
)

logger.data_loaded(len(train_loader.dataset))
print(f"\nDataset: {len(train_loader.dataset):,} samples, {len(train_loader)} batches")

In [ ]:
# Optional: Visualize dataset samples
if CONFIG['visualize']:
    logger.info("📸 Visualizing dataset samples...")
    visualize_samples(train_loader.dataset, num_samples=16)

## Training

In [ ]:
# Create universal trainer
trainer = UniversalTrainer(
    model=model,
    model_name="ResNet18",
    dataset_name="CIFAR-100",
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'], 
    learning_rate=CONFIG['learning_rate'],
    optimizer_name="SGD",  # Best for ResNet18
    use_mlflow=CONFIG['use_mlflow'],
    # Additional parameters for comprehensive tracking
    input_size=(3, 32, 32),
    use_pretrained=True,
    train_size=len(train_loader.dataset),
    momentum=0.9,
    weight_decay=4e-5
)

logger.info("Trainer initialized - ready for training!")

In [ ]:
# Start training with comprehensive monitoring
logger.info("Starting ResNet18 training on CIFAR-100")
print("\n" + "="*60)
print(" STARTING RESNET18 TRAINING")
print("="*60)

# Train the model
summary = trainer.train(train_loader)

print("\n" + "="*60) 
print(" TRAINING COMPLETED!")
print("="*60)

## Training Results & Analysis

In [ ]:
# Display final results
if summary:
    print("\nFINAL TRAINING SUMMARY:")
    print("="*40)
    print(f"Status: {summary.get('status', 'Unknown')}")
    print(f"Total Time: {summary.get('total_time_minutes', 0):.1f} minutes")
    print(f"Best Accuracy: {summary.get('best_accuracy', 0):.4f}")
    print(f"Best Loss: {summary.get('best_loss', float('inf')):.4f}")
    print(f"Epochs: {summary.get('epochs_completed', 0)}/{CONFIG['epochs']}")
    print("="*40)
    
    # Calculate performance metrics
    if summary.get('total_time_minutes', 0) > 0:
        samples_per_min = len(train_loader.dataset) * summary['epochs_completed'] / summary['total_time_minutes']
        print(f"Performance: {samples_per_min:,.0f} samples/minute")

logger.info("ResNet18-CIFAR100 experiment completed!")